# AlphaGo Zero for connect 2

https://web.stanford.edu/~surag/posts/alphazero.html <br>
https://www.youtube.com/watch?v=62nq4Zsn8vc&ab_channel=JoshVarty <br>

### Value network and self play
1) Play a bunch of games against yourself and record whether a given board state eventually lead to a win or a loss. We take a snapshot of what the board looks like when presented to each player and want to determine how good of a position they are currently in before they take a move. <br>

`
 BOARD STATE 
[ 0  0  0  0] # Start of the game, presented to Player 1. Player 1 plays in the first position.
[-1  0  0  0] # Player 2 is presented with this board. Player 2 plays in the third position.
[ 1  0 -1  0] # Player 1 is presented with this board. Player 1 plays in the second position
[-1 -1  1  0] # Player 2 is presented with this board. Player 2 was lost as Player 1 can connect 2
`

2) Create a labelled training set for the value NN by going back through all of the game states held by Player 1 and marking them with a reward of 1. Similarily go through all of the states held by Player 2 and mark them with a reward of -1. <br>
` 
  BOARD STATE   RES 
([ 0  0  0  0], 1) # Player 1 
([-1  0  0  0],-1) # Player 2 
([ 1  0 -1  0], 1) # Player 1 
([-1 -1  1  0],-1) # Player 2 
`

3) After completing thousands of self play games, we shuffle up the board training data and feed it into our neural network to train it to recognise what the probability of winning is given a particular board state. The output will be from -1 to 1. This is essentially an image recognition task, which is why AlphaGo uses resnets.

<img src="value_network.PNG" alt="drawing" width="400"/> 


### Policy Network
Takes a game state / board as input and outputs next move a probability distribution of next moves, with move with the highest probability being the move which the policy network thinks will do the best.

<img src="policy_network.PNG" alt="drawing" width="400"/> 

Based upon the output above, we will win 88% of the time if you make a move into the first position and have a 10% probability of winning in the last position. Wierdly, it outputs a non-zero chance of winning if we take invalid moves where there are already pieces. This is because we cannot control the NN to output 0 values for invalid moves. Instead, we will have to write code to mask out invalid moves and redistribute the values.

The way we train the network is to actually encourage it to give the same output an Monte Carlo Tree search (more on that later). To do this we create a dataset that includes a board state, and also what the MCTS suggested we do at that position.
`
BOARD STATE             MCTS        
([ 0  0  0  0], [0.1, 0.4, 0.4, 0.1]) 
([-1  0  0  0], [0.0, 0.3, 0.3, 0.3]) 
([ 1  0 -1  0], [0.0, 0.8, 0.0, 0.2]) 
`

There is actually an interesting circular dependency, where by we are training our policy network to output results that match the MCTS, but we actually use our policy to network to choose the most promising next positions to expolore with MCTS. In theory, improving the policy network will improve the MCTS and visa versa.


### AlphaGo Zero actually uses the same network to output the policy and value

<img src="policy_value.PNG" alt="drawing" width="400"/> 

### MCTS
Normally MCTS would entail doing rollouts until the a particular game is ran to conclusion and then the result is backed up to update the previous states; however, with AlphaGo Zero we actually use the results from the value network to score each state. Of course, if we do happen to reach a terminal game state, then the actual result will be used instead and backed up. The lack of focus of complete rollouts can be hugely befefitial in a game like Go where there are any moves until termination.

In [ ]:
import torch

from game import Connect2Game
from model import Connect2Model
from trainer import Trainer

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = {
    'batch_size': 64,
    'num_simulations': 100,                 # Number of Monte Carlo simulations for each move
    'numIters': 500,                        # Total number of training iterations
    'numEps': 100,                          # Number of full games (episodes) to run during each iteration
    'numItersForTrainExamplesHistory': 20,
    'epochs': 2,                            # Number of epochs of training per iteration
    'checkpoint_path': 'latest.pth'         # location to save latest set of weights
}

game = Connect2Game()
board_size = game.get_board_size()
action_size = game.get_action_size()

model = Connect2Model(board_size, action_size, device)

trainer = Trainer(game, model, args)
trainer.learn()

1/500

Policy Loss 1.4157464504241943
Value Loss 0.8966259807348251
Examples:
tensor([0.2723, 0.2117, 0.2783, 0.2376], device='cuda:0')
tensor([0.0400, 0.0100, 0.9300, 0.0200], device='cuda:0')

Policy Loss 1.4188497811555862
Value Loss 0.8844451904296875
Examples:
tensor([0.2717, 0.2126, 0.2786, 0.2370], device='cuda:0')
tensor([0.0400, 0.0100, 0.9300, 0.0200], device='cuda:0')
2/500

Policy Loss 1.4166783094406128
Value Loss 0.871482715010643
Examples:
tensor([0.2666, 0.2081, 0.2799, 0.2454], device='cuda:0')
tensor([0.2600, 0.3400, 0.0000, 0.4000], device='cuda:0')

Policy Loss 1.411540761590004
Value Loss 0.8693072274327278
Examples:
tensor([0.2656, 0.2087, 0.2807, 0.2450], device='cuda:0')
tensor([0.2600, 0.3400, 0.0000, 0.4000], device='cuda:0')
3/500

Policy Loss 1.4092235565185547
Value Loss 0.8500819355249405
Examples:
tensor([0.2764, 0.2219, 0.2709, 0.2309], device='cuda:0')
tensor([0.0500, 0.9500, 0.0000, 0.0000], device='cuda:0')

Policy Loss 1.404719591140747
Value Loss 0.

tensor([0.2400, 0.3700, 0.0000, 0.3900], device='cuda:0')

Policy Loss 1.2755832523107529
Value Loss 0.4786642976105213
Examples:
tensor([0.2329, 0.3019, 0.2686, 0.1966], device='cuda:0')
tensor([0.0400, 0.9600, 0.0000, 0.0000], device='cuda:0')
23/500

Policy Loss 1.2757640182971954
Value Loss 0.4857849106192589
Examples:
tensor([0.2135, 0.2235, 0.3059, 0.2571], device='cuda:0')
tensor([0.2400, 0.3700, 0.0000, 0.3900], device='cuda:0')

Policy Loss 1.2745856046676636
Value Loss 0.4816209636628628
Examples:
tensor([0.2282, 0.3114, 0.2674, 0.1930], device='cuda:0')
tensor([0.0400, 0.9600, 0.0000, 0.0000], device='cuda:0')
24/500

Policy Loss 1.275465726852417
Value Loss 0.4746265709400177
Examples:
tensor([0.2260, 0.3167, 0.2661, 0.1913], device='cuda:0')
tensor([0.0400, 0.9600, 0.0000, 0.0000], device='cuda:0')

Policy Loss 1.2649015486240387
Value Loss 0.46696967631578445
Examples:
tensor([0.2236, 0.3219, 0.2650, 0.1895], device='cuda:0')
tensor([0.0400, 0.9600, 0.0000, 0.0000], devic

tensor([0.1553, 0.2762, 0.3526, 0.2159], device='cuda:0')
tensor([0.0100, 0.0100, 0.9700, 0.0100], device='cuda:0')
44/500

Policy Loss 1.0835772156715393
Value Loss 0.13969111070036888
Examples:
tensor([0.1538, 0.2475, 0.2658, 0.3329], device='cuda:0')
tensor([0.1800, 0.4000, 0.0000, 0.4200], device='cuda:0')

Policy Loss 1.0874934494495392
Value Loss 0.13948346488177776
Examples:
tensor([0.1525, 0.2499, 0.2632, 0.3344], device='cuda:0')
tensor([0.1800, 0.4000, 0.0000, 0.4200], device='cuda:0')
45/500

Policy Loss 1.3262956142425537
Value Loss 0.14889003708958626
Examples:
tensor([0.1200, 0.3230, 0.3406, 0.2163], device='cuda:0')
tensor([0.0400, 0.0000, 0.0000, 0.9600], device='cuda:0')

Policy Loss 1.323670208454132
Value Loss 0.14345251396298409
Examples:
tensor([0.1500, 0.2498, 0.2574, 0.3427], device='cuda:0')
tensor([0.1800, 0.4100, 0.0000, 0.4100], device='cuda:0')
46/500

Policy Loss 1.0784142911434174
Value Loss 0.1206500343978405
Examples:
tensor([0.1188, 0.4537, 0.2540, 0.17

tensor([0.1100, 0.4400, 0.0000, 0.4500], device='cuda:0')
65/500

Policy Loss 0.825343668460846
Value Loss 0.020335307344794273
Examples:
tensor([0.1009, 0.3166, 0.1677, 0.4148], device='cuda:0')
tensor([0.1100, 0.4400, 0.0000, 0.4500], device='cuda:0')

Policy Loss 0.8185189366340637
Value Loss 0.01953367900568992
Examples:
tensor([0.1000, 0.3193, 0.1648, 0.4159], device='cuda:0')
tensor([0.1100, 0.4400, 0.0000, 0.4500], device='cuda:0')
66/500

Policy Loss 1.3846348822116852
Value Loss 0.016436856472864747
Examples:
tensor([0.0831, 0.3372, 0.4037, 0.1759], device='cuda:0')
tensor([0.0100, 0.0100, 0.9700, 0.0100], device='cuda:0')

Policy Loss 1.3449247032403946
Value Loss 0.018009663908742368
Examples:
tensor([0.0985, 0.3174, 0.1582, 0.4259], device='cuda:0')
tensor([0.1000, 0.4500, 0.0000, 0.4500], device='cuda:0')
67/500

Policy Loss 0.8449588119983673
Value Loss 0.019462269730865955
Examples:
tensor([0.0814, 0.3330, 0.4078, 0.1778], device='cuda:0')
tensor([0.0100, 0.0100, 0.9700,

Policy Loss 0.5906958281993866
Value Loss 0.0033229757682420313
Examples:
tensor([0.0591, 0.3616, 0.0776, 0.5017], device='cuda:0')
tensor([0.0600, 0.3900, 0.0000, 0.5500], device='cuda:0')

Policy Loss 0.5798753648996353
Value Loss 0.0033900798007380217
Examples:
tensor([0.0358, 0.2723, 0.5837, 0.1081], device='cuda:0')
tensor([0.0100, 0.0100, 0.9700, 0.0100], device='cuda:0')
87/500

Policy Loss 0.5913925766944885
Value Loss 0.0035384841030463576
Examples:
tensor([0.0050, 0.8672, 0.1042, 0.0236], device='cuda:0')
tensor([0.0100, 0.9900, 0.0000, 0.0000], device='cuda:0')

Policy Loss 0.5802256762981415
Value Loss 0.003283182595623657
Examples:
tensor([0.0342, 0.2621, 0.5994, 0.1043], device='cuda:0')
tensor([0.0100, 0.0100, 0.9700, 0.0100], device='cuda:0')
88/500

Policy Loss 0.5414046496152878
Value Loss 0.003058533649891615
Examples:
tensor([0.0046, 0.8748, 0.0991, 0.0215], device='cuda:0')
tensor([0.0100, 0.9900, 0.0000, 0.0000], device='cuda:0')

Policy Loss 0.5326973311603069
Va

...

tensor([1.9997e-02, 6.1412e-03, 4.0708e-06, 9.7386e-01], device='cuda:0')
tensor([0.0200, 0.0000, 0.0000, 0.9800], device='cuda:0')
423/500

Policy Loss 0.2768261879682541
Value Loss 8.164174891689413e-11
Examples:
tensor([9.8943e-03, 6.0236e-01, 1.1910e-08, 3.8774e-01], device='cuda:0')
tensor([0.0100, 0.6000, 0.0000, 0.3900], device='cuda:0')

Policy Loss 0.28670370765030384
Value Loss 7.661431905559013e-11
Examples:
tensor([9.5653e-03, 2.0170e-05, 9.9004e-01, 3.7603e-04], device='cuda:0')
tensor([0.0100, 0.0000, 0.9900, 0.0000], device='cuda:0')
424/500

Policy Loss 0.2865273803472519
Value Loss 6.445102784802259e-11
Examples:
tensor([1.0117e-02, 5.9809e-01, 1.1272e-08, 3.9180e-01], device='cuda:0')
tensor([0.0100, 0.6000, 0.0000, 0.3900], device='cuda:0')

Policy Loss 0.29496878385543823
Value Loss 6.087056553250036e-11
Examples:
tensor([1.9992e-02, 3.4256e-03, 4.4185e-06, 9.7658e-01], device='cuda:0')
tensor([0.0200, 0.0000, 0.0000, 0.9800], device='cuda:0')
425/500

Policy Loss 0

Policy Loss 0.5723298862576485
Value Loss 7.234836341130091e-11
Examples:
tensor([2.0187e-02, 4.3983e-01, 6.0181e-11, 5.3998e-01], device='cuda:0')
tensor([0.0200, 0.9800, 0.0000, 0.0000], device='cuda:0')

Policy Loss 0.5403863899409771
Value Loss 7.695776654825792e-11
Examples:
tensor([1.9552e-02, 4.8277e-01, 5.7774e-11, 4.9768e-01], device='cuda:0')
tensor([0.0200, 0.9800, 0.0000, 0.0000], device='cuda:0')
443/500

Policy Loss 0.46711229532957077
Value Loss 8.461548128835261e-11
Examples:
tensor([1.8641e-02, 5.2574e-01, 5.5237e-11, 4.5562e-01], device='cuda:0')
tensor([0.0100, 0.9900, 0.0000, 0.0000], device='cuda:0')

Policy Loss 0.4725724346935749
Value Loss 8.348943758562655e-11
Examples:
tensor([1.0068e-02, 3.8892e-01, 1.6358e-08, 6.0101e-01], device='cuda:0')
tensor([0.0100, 0.3900, 0.0000, 0.6000], device='cuda:0')
444/500

Policy Loss 0.4447357580065727
Value Loss 9.056486116598705e-11
Examples:
tensor([9.8093e-03, 1.5724e-05, 9.8999e-01, 1.8900e-04], device='cuda:0')
tensor(

tensor([0.0100, 0.0000, 0.9900, 0.0000], device='cuda:0')

Policy Loss 0.3101510629057884
Value Loss 2.2258158510757298e-10
Examples:
tensor([9.4806e-03, 5.0006e-01, 1.1868e-08, 4.9046e-01], device='cuda:0')
tensor([0.0100, 0.5000, 0.0000, 0.4900], device='cuda:0')
462/500

Policy Loss 0.2965414747595787
Value Loss 2.1910594172336317e-10
Examples:
tensor([4.3950e-02, 2.8774e-02, 9.1283e-07, 9.2728e-01], device='cuda:0')
tensor([0.0200, 0.0000, 0.0000, 0.9800], device='cuda:0')

Policy Loss 0.30445992946624756
Value Loss 2.1152440909943948e-10
Examples:
tensor([9.7404e-03, 2.1032e-05, 9.9002e-01, 2.1693e-04], device='cuda:0')
tensor([0.0100, 0.0000, 0.9900, 0.0000], device='cuda:0')
463/500

Policy Loss 0.3055267930030823
Value Loss 2.2094949481132886e-10
Examples:
tensor([9.8699e-03, 4.9491e-01, 1.3299e-08, 4.9522e-01], device='cuda:0')
tensor([0.0100, 0.5000, 0.0000, 0.4900], device='cuda:0')

Policy Loss 0.3030960150063038
Value Loss 2.1008893930085648e-10
Examples:
tensor([2.9725e-0

tensor([1.0264e-02, 4.6068e-01, 6.2131e-09, 5.2906e-01], device='cuda:0')
tensor([0.0100, 0.4600, 0.0000, 0.5300], device='cuda:0')
481/500

Policy Loss 0.7131358981132507
Value Loss 5.842143435685898e-11
Examples:
tensor([1.0085e-02, 4.6626e-01, 6.3406e-09, 5.2365e-01], device='cuda:0')
tensor([0.0100, 0.4600, 0.0000, 0.5300], device='cuda:0')

Policy Loss 0.6624820530414581
Value Loss 6.438731492419691e-11
Examples:
tensor([1.0350e-02, 4.5945e-01, 7.1707e-09, 5.3020e-01], device='cuda:0')
tensor([0.0100, 0.4600, 0.0000, 0.5300], device='cuda:0')
482/500

Policy Loss 0.610584169626236
Value Loss 6.753231307499163e-11
Examples:
tensor([1.0056e-02, 4.6483e-01, 7.0845e-09, 5.2511e-01], device='cuda:0')
tensor([0.0100, 0.4600, 0.0000, 0.5300], device='cuda:0')

Policy Loss 0.5803443863987923
Value Loss 7.301469845399922e-11
Examples:
tensor([9.8711e-03, 1.2071e-05, 9.8996e-01, 1.6137e-04], device='cuda:0')
tensor([0.0100, 0.0000, 0.9900, 0.0000], device='cuda:0')
483/500

Policy Loss 0.53

Policy Loss 0.3267146348953247
Value Loss 1.4201653741885423e-10
Examples:
tensor([9.8153e-03, 5.4538e-01, 8.0640e-09, 4.4480e-01], device='cuda:0')
tensor([0.0100, 0.5500, 0.0000, 0.4400], device='cuda:0')

Policy Loss 0.3204425051808357
Value Loss 1.4065287823328276e-10
Examples:
tensor([9.5000e-03, 5.5054e-01, 7.5843e-09, 4.3996e-01], device='cuda:0')
tensor([0.0100, 0.5500, 0.0000, 0.4400], device='cuda:0')
